# Prototype aligning Fairlay matches/series with predictions.

## Setup.

In [1]:
import os
os.chdir("..")

In [260]:
import gzip
import json
import numpy as np
import pandas as pd

import src.load
import src.stats

In [303]:
import importlib
importlib.reload(src.stats)

<module 'src.stats' from '/Users/yl3/github/dota2/src/stats.py'>

In [144]:
pd.set_option('max_rows', 6)
pd.set_option('max_columns', 50)

## Load data.

### Match predictions

In [31]:
with gzip.open('data/raw/premium_matches.2019-08-19.json.gz', 'rb') as fh:
    matches = src.load.MatchDF.from_json(json.load(fh)['data'])
matches.df

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,radiant_players,dire_valveId,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series
matchId,,,,,,,,,,,,,,,,,
3805456168,2018-03-29 09:44:13,Dota 2 Asia Championships 2018,INVICTUS GAMING,Mineski,False,"[Xxs, Agressif, Q, Srf, BoBoKa]","[iceiceice, Mushi, ninjaboogie, Jabz, Moon]",-3805456168,2779,5,"[129958758, 130416036, 140153524, 156662698, 2...",543897,"[84772440, 89871557, 91443418, 100471531, 1134...",9643,1522316653000,1522316653000,0
3805522795,2018-03-29 10:37:41,Dota 2 Asia Championships 2018,OpTic Gaming,Newbee,True,"[zai, 33, ppd, Pajkatt, CCnC]","[Faith, kpii, Moogy, kaka, Sccc]",-3805522795,2313,5026801,"[73562326, 86698277, 86727555, 87285329, 22166...",1375614,"[82327674, 87012746, 117281554, 139876032, 149...",9643,1522319861000,1522319861000,0
3805545412,2018-03-29 10:54:54,Dota 2 Asia Championships 2018,Virtus.pro,paiN Gaming,True,"[9pasha, No[o]ne, RAMZES666, Solo, RodjER]","[Kingrd, hFn, Arms, Duster, tavo]",-3805545412,1759,1883502,"[92423451, 106573901, 132851371, 134556694, 15...",67,"[84853828, 94004717, 101062921, 116525052, 117...",9643,1522320894000,1522320894000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973241356,2019-08-18 06:16:02,The International 2019,Vici Gaming,OG,True,"[Ori, Paparazi, Yang, Dy, Fade]","[n0tail, JerAx, 7ckngMad, topson, ana]",359053,2372,726228,"[107803494, 137193239, 139937922, 143693439, 1...",2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",10749,1566108962000,1566108962000,0
4973304079,2019-08-18 06:59:41,The International 2019,Ninjas in Pyjamas,Evil Geniuses,False,"[33, ppd, Fata, Ace, Saksa]","[Cr1t-, s4, Arteezy, Fly, SumaiL]",359048,2255,6214973,"[86698277, 86727555, 86799300, 97590558, 10373...",39,"[25907144, 41231571, 86745912, 94155156, 11162...",10749,1566111581000,1566107488000,1
4973348952,2019-08-18 07:28:25,The International 2019,OG,Vici Gaming,True,"[n0tail, JerAx, 7ckngMad, topson, ana]","[Ori, Paparazi, Yang, Dy, Fade]",359053,2312,2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",726228,"[107803494, 137193239, 139937922, 143693439, 1...",10749,1566113305000,1566108962000,1


In [32]:
skills_mat = pd.read_csv('fitted/ti9_grp_new_params_by_map.player_skills.gz',
                         index_col=0, sep='\t')
skills_mat

,1296625,3916428,3940262,4281729,5150808,6103402,6922000,9403474,10366616,11550182,...,858925382,864320791,869108920,876110702,879538479,882896652,893135754,907097333,907351651,909485668
matchId,,,,,,,,,,,,,,,,,,,,,
3805456168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3805522795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3805545412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973205314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4973241356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4973348952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
pred_df = pd.read_csv('fitted/ti9_grp_new_params_by_map.win_probs.gz',
                      index_col=0, sep='\t')
pred_df

,dire_skill,dire_skill_sd,iter,pred_win_prob,pred_win_prob_unknown_side,radi_adv,radi_skill,radi_skill_sd,startTimestamp,win_prob+2sd,win_prob-2sd
matchId,,,,,,,,,,,
3805456168,4.294731,NaN,0,0.296599,0.262568,0.529562,1.174532,NaN,NaN,NaN,NaN
3805522795,2.340950,NaN,0,0.647569,0.605528,0.529562,3.636498,NaN,NaN,NaN,NaN
3805545412,-0.141867,NaN,0,0.874645,0.852605,0.529562,5.156587,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4973205314,4.187800,0.028375,142,0.358134,0.321125,0.510991,1.926388,0.028473,1.566107e+12,0.364317,0.351997
4973241356,5.653038,0.133046,143,0.488806,0.446811,0.510982,5.007709,0.059549,1.566109e+12,0.513095,0.464570
4973348952,5.186006,0.033490,144,0.565414,0.522908,0.512399,5.463105,0.033490,1.566113e+12,0.573156,0.557640


In [304]:
match_pred = src.stats.MatchPred(matches, pred_df, 3.0, skills_mat)
match_pred.df

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,radiant_players,dire_valveId,dire_players,league_id,startTimestamp_x,series_start_time,match_i_in_series,dire_skill,dire_skill_sd,iter,pred_win_prob,pred_win_prob_unknown_side,radi_adv,radi_skill,radi_skill_sd,startTimestamp_y,win_prob+2sd,win_prob-2sd
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3805456168,2018-03-29 09:44:13,Dota 2 Asia Championships 2018,INVICTUS GAMING,Mineski,False,"[Xxs, Agressif, Q, Srf, BoBoKa]","[iceiceice, Mushi, ninjaboogie, Jabz, Moon]",-3805456168,2779,5,"[129958758, 130416036, 140153524, 156662698, 2...",543897,"[84772440, 89871557, 91443418, 100471531, 1134...",9643,1522316653000,1522316653000,0,4.294731,NaN,0,0.296599,0.262568,0.529562,1.174532,NaN,NaN,NaN,NaN
3805522795,2018-03-29 10:37:41,Dota 2 Asia Championships 2018,OpTic Gaming,Newbee,True,"[zai, 33, ppd, Pajkatt, CCnC]","[Faith, kpii, Moogy, kaka, Sccc]",-3805522795,2313,5026801,"[73562326, 86698277, 86727555, 87285329, 22166...",1375614,"[82327674, 87012746, 117281554, 139876032, 149...",9643,1522319861000,1522319861000,0,2.340950,NaN,0,0.647569,0.605528,0.529562,3.636498,NaN,NaN,NaN,NaN
3805545412,2018-03-29 10:54:54,Dota 2 Asia Championships 2018,Virtus.pro,paiN Gaming,True,"[9pasha, No[o]ne, RAMZES666, Solo, RodjER]","[Kingrd, hFn, Arms, Duster, tavo]",-3805545412,1759,1883502,"[92423451, 106573901, 132851371, 134556694, 15...",67,"[84853828, 94004717, 101062921, 116525052, 117...",9643,1522320894000,1522320894000,0,-0.141867,NaN,0,0.874645,0.852605,0.529562,5.156587,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973241356,2019-08-18 06:16:02,The International 2019,Vici Gaming,OG,True,"[Ori, Paparazi, Yang, Dy, Fade]","[n0tail, JerAx, 7ckngMad, topson, ana]",359053,2372,726228,"[107803494, 137193239, 139937922, 143693439, 1...",2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",10749,1566108962000,1566108962000,0,5.653038,0.133046,143,0.488806,0.446811,0.510982,5.007709,0.059549,1.566109e+12,0.513095,0.464570
4973304079,2019-08-18 06:59:41,The International 2019,Ninjas in Pyjamas,Evil Geniuses,False,"[33, ppd, Fata, Ace, Saksa]","[Cr1t-, s4, Arteezy, Fly, SumaiL]",359048,2255,6214973,"[86698277, 86727555, 86799300, 97590558, 10373...",39,"[25907144, 41231571, 86745912, 94155156, 11162...",10749,1566111581000,1566107488000,1,4.186484,0.065874,141,0.358223,0.321211,0.510962,1.926270,0.066605,1.566112e+12,0.372705,0.343996
4973348952,2019-08-18 07:28:25,The International 2019,OG,Vici Gaming,True,"[n0tail, JerAx, 7ckngMad, topson, ana]","[Ori, Paparazi, Yang, Dy, Fade]",359053,2312,2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",726228,"[107803494, 137193239, 139937922, 143693439, 1...",10749,1566113305000,1566108962000,1,5.186006,0.033490,144,0.565414,0.522908,0.512399,5.463105,0.033490,1.566113e+12,0.573156,0.557640


### Odds from Fairlay

Only keep the earliest available odds of individual maps.

In [9]:
fl = pd.read_pickle("data/external/dota2_markets.20190827.pkl")
with pd.option_context('max_rows', 50):
    display(fl.iloc[0].T)

Comp                                       Dota 2 - Destiny Cup
Title                              Team Drinking vs. FrozenDawn
Descr                                                     Match
dota_market_type                                          other
LastSoftCh                            2019-08-12 22:51:15-04:00
ClosD                                 2019-08-13 00:00:00-04:00
wager_type                                              against
RunnerName                                        Team Drinking
handicap                                                      0
odds                                                      1.649
amount                                                    24.11
winp                                                     0.3937
winp_c                                                   0.3931
RunnerVolMatched                                              0
CatID                                                        32
SettlD                                20

In [308]:
sel = (fl.Comp == 'Dota 2 - The International') & (fl.dota_market_type == 'map')
fl_f = (fl.loc[sel].sort_values('LastCh')
        .groupby(['ID', 'wager_type']).apply(lambda d: d.iloc[0]))
fl_f

Comp                     Title  \
ID           wager_type                                                         
177218520623 against     Dota 2 - The International  Team Secret vs. Alliance   
             on          Dota 2 - The International  Team Secret vs. Alliance   
177218529861 against     Dota 2 - The International  Team Secret vs. Alliance   
...                                             ...                       ...   
178196223735 on          Dota 2 - The International        OG vs. Team Liquid   
178199751032 against     Dota 2 - The International        OG vs. Team Liquid   
             on          Dota 2 - The International        OG vs. Team Liquid   

                           Descr dota_market_type                LastSoftCh  \
ID           wager_type                                                       
177218520623 against     1st Map              map 2019-08-14 04:17:17-04:00   
             on          1st Map              map 2019-08-13 23:51:32-04:00   
177218529861 against     2nd Map              map 2019-08-14 04:17:17-04:00   
...                          ...              ...                       ...   
178196223735 on          3rd Map              map 2019-08-25 07:00:00-04:00   
178199751032 against     4th Map              map 2019-08-25 07:59:59-04:00   
             on          4th Map              map 2019-08-25 07:59:59-04:00   

                                            ClosD wager_type   RunnerName  \
ID           wager_type                                                     
177218520623 against    2019-08-14 20:59:51-04:00    against  Team Secret   
             on         2019-08-14 20:59:51-04:00         on  Team Secret   
177218529861 against    2019-08-14 20:59:51-04:00    against  Team Secret   
...                                           ...        ...          ...   
178196223735 on         2019-08-25 07:27:03-04:00         on           OG   
178199751032 against    2019-08-25 08:25:50-04:00    against           OG   
             on         2019-08-25 08:25:50-04:00         on           OG   

                         handicap   odds  amount    winp  winp_c  \
ID           wager_type                                            
177218520623 against          0.0  3.538   75.11  0.7174  0.7169   
             on               0.0  1.414   69.11  0.7072  0.7077   
177218529861 against          0.0  3.564   76.11  0.7194  0.7189   
...                           ...    ...     ...     ...     ...   
178196223735 on               0.0  1.751  638.11  0.5711  0.5717   
178199751032 against          0.0  2.479  710.11  0.5967  0.5961   
             on               0.0  1.499  960.11  0.6671  0.6677   

                         RunnerVolMatched  CatID                    SettlD  \
ID           wager_type                                                      
177218520623 against                  0.0     32 2019-08-14 21:59:51-04:00   
             on                       0.0     32 2019-08-14 21:59:51-04:00   
177218529861 against                  0.0     32 2019-08-14 21:59:51-04:00   
...                                   ...    ...                       ...   
178196223735 on                       0.0     32 2019-08-25 07:33:03-04:00   
178199751032 against                  0.0     32 2019-08-25 08:31:50-04:00   
             on                       0.0     32 2019-08-25 08:31:50-04:00   

                         Status  _Type  _Period  SettlT    Comm  \
ID           wager_type                                           
177218520623 against          0      0        0       0  0.0025   
             on               0      0        0       0  0.0025   
177218529861 against          0      0        0       0  0.0025   
...                         ...    ...      ...     ...     ...   
178196223735 on               1      0        0       0  0.0025   
178199751032 against          1      0        0       0  0.0025   
             on               1      0        0       0  0.

In [309]:
fl_f.Title.str.extract(pat=r'^(.+) vs. (.+)$')

0            1
ID           wager_type                          
177218520623 against     Team Secret     Alliance
             on          Team Secret     Alliance
177218529861 against     Team Secret     Alliance
...                              ...          ...
178196223735 on                   OG  Team Liquid
178199751032 against              OG  Team Liquid
             on                   OG  Team Liquid

[808 rows x 2 columns]

In [310]:
fl_f.Descr.str[0].value_counts()

1    392
2    360
3     50
4      4
5      2
Name: Descr, dtype: int64

## Test a class for matching Fairlay odds with predictions.

In [311]:
with pd.option_context('max_columns', 30):
    display(match_pred.df.query('iter > 0'))

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,radiant_players,dire_valveId,dire_players,league_id,startTimestamp_x,series_start_time,match_i_in_series,dire_skill,dire_skill_sd,iter,pred_win_prob,pred_win_prob_unknown_side,radi_adv,radi_skill,radi_skill_sd,startTimestamp_y,win_prob+2sd,win_prob-2sd
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4967600837,2019-08-15 02:01:07,The International 2019,KEEN GAMING,TNC Predator,False,"[old eLeVeN, old chicken, kaka, --, dark]","[Gabbi, TIMS, Armel, Eyyou, Kuku]",358208,3193,2626685,"[134276083, 135878232, 139876032, 255219872, 3...",2108395,"[152545459, 155494381, 164532005, 173476224, 1...",10749,1565834467000,1565834467000,0,3.721261,0.035105,2,0.381560,0.342069,0.528181,1.744280,0.035113,1.565834e+12,0.389401,0.373780
4967601481,2019-08-15 02:01:58,The International 2019,PSG.LGD,Chaos Esports Club,True,"[xNova, Chalice, fy, Somnus丶M, Ame]","[MATUMBAMAN, MISERY, MiLAN, vtFaded, KheZu]",358209,1667,15,"[94296097, 94738847, 101695162, 106863163, 177...",7203342,"[72312627, 87382579, 98172857, 142139318, 1690...",10749,1565834518000,1565834518000,0,2.267840,0.802255,3,0.721976,0.684065,0.528165,4.602535,0.961561,1.565835e+12,0.856822,0.529822
4967601727,2019-08-15 02:02:17,The International 2019,Team Liquid,Newbee,False,"[MinD_ContRoL, KuroKy, w33, GH, Miracle]","[pieliedie, Sneyking, MSS, YawaR, CCnC]",358210,2873,2163,"[34505203, 82262664, 86700461, 101356886, 1052...",6214538,"[6922000, 10366616, 86726887, 108452107, 22166...",10749,1565834537000,1565834537000,0,3.414739,0.797336,5,0.644085,0.602035,0.528062,4.666100,0.949415,1.565835e+12,0.805289,0.441913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973241356,2019-08-18 06:16:02,The International 2019,Vici Gaming,OG,True,"[Ori, Paparazi, Yang, Dy, Fade]","[n0tail, JerAx, 7ckngMad, topson, ana]",359053,2372,726228,"[107803494, 137193239, 139937922, 143693439, 1...",2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",10749,1566108962000,1566108962000,0,5.653038,0.133046,143,0.488806,0.446811,0.510982,5.007709,0.059549,1.566109e+12,0.513095,0.464570
4973304079,2019-08-18 06:59:41,The International 2019,Ninjas in Pyjamas,Evil Geniuses,False,"[33, ppd, Fata, Ace, Saksa]","[Cr1t-, s4, Arteezy, Fly, SumaiL]",359048,2255,6214973,"[86698277, 86727555, 86799300, 97590558, 10373...",39,"[25907144, 41231571, 86745912, 94155156, 11162...",10749,1566111581000,1566107488000,1,4.186484,0.065874,141,0.358223,0.321211,0.510962,1.926270,0.066605,1.566112e+12,0.372705,0.343996
4973348952,2019-08-18 07:28:25,The International 2019,OG,Vici Gaming,True,"[n0tail, JerAx, 7ckngMad, topson, ana]","[Ori, Paparazi, Yang, Dy, Fade]",359053,2312,2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",726228,"[107803494, 137193239, 139937922, 143693439, 1...",10749,1566113305000,1566108962000,1,5.186006,0.033490,144,0.565414,0.522908,0.512399,5.463105,0.033490,1.566113e+12,0.573156,0.557640


In [312]:
fl_f.head()

Comp                     Title  \
ID           wager_type                                                         
177218520623 against     Dota 2 - The International  Team Secret vs. Alliance   
             on          Dota 2 - The International  Team Secret vs. Alliance   
177218529861 against     Dota 2 - The International  Team Secret vs. Alliance   
             on          Dota 2 - The International  Team Secret vs. Alliance   
177218614562 against     Dota 2 - The International    Team Liquid vs. Newbee   

                           Descr dota_market_type                LastSoftCh  \
ID           wager_type                                                       
177218520623 against     1st Map              map 2019-08-14 04:17:17-04:00   
             on          1st Map              map 2019-08-13 23:51:32-04:00   
177218529861 against     2nd Map              map 2019-08-14 04:17:17-04:00   
             on          2nd Map              map 2019-08-14 08:29:26-04:00   
177218614562 against     1st Map              map 2019-08-14 04:17:21-04:00   

                                            ClosD wager_type   RunnerName  \
ID           wager_type                                                     
177218520623 against    2019-08-14 20:59:51-04:00    against  Team Secret   
             on         2019-08-14 20:59:51-04:00         on  Team Secret   
177218529861 against    2019-08-14 20:59:51-04:00    against  Team Secret   
             on         2019-08-14 20:59:51-04:00         on  Team Secret   
177218614562 against    2019-08-14 20:59:54-04:00    against  Team Liquid   

                         handicap   odds  amount    winp  winp_c  \
ID           wager_type                                            
177218520623 against          0.0  3.538   75.11  0.7174  0.7169   
             on               0.0  1.414   69.11  0.7072  0.7077   
177218529861 against          0.0  3.564   76.11  0.7194  0.7189   
             on               0.0  1.294   97.11  0.7728  0.7732   
177218614562 against          0.0  3.358   70.11  0.7022  0.7017   

                         RunnerVolMatched  CatID                    SettlD  \
ID           wager_type                                                      
177218520623 against                  0.0     32 2019-08-14 21:59:51-04:00   
             on                       0.0     32 2019-08-14 21:59:51-04:00   
177218529861 against                  0.0     32 2019-08-14 21:59:51-04:00   
             on                       0.0     32 2019-08-14 21:59:51-04:00   
177218614562 against                  0.0     32 2019-08-14 21:59:54-04:00   

                         Status  _Type  _Period  SettlT    Comm  \
ID           wager_type                                           
177218520623 against          0      0        0       0  0.0025   
             on               0      0        0       0  0.0025   
177218529861 against          0      0        0       0  0.0025   
             on               0      0        0       0  0.0025   
177218614562 against          0      0        0       0  0.0025   

                                                             Settler  \
ID           wager_type                                                
177218520623 against     {'1': True, '777889': True, '777555': True}   
             on          {'1': True, '777889': True, '777555': True}   
177218529861 against     {'1': True, '777889': True, '777555': True}   
             on          {'1': True, '777889': True, '777555': True}   
177218614562 against     {'1': True, '777889': True, '777555': True}   

                                                 ComRecip  MinVal  MaxVal  \
ID           wager_type                                                     
177218520623 against     {'1011849': 0.5, '1013506': 0.5}     0.0     0.0   
             on          {'1011849': 0.5, '1013506': 0.5}     0.0     0.0   
177218529861 against     {'1011849': 0.5, '1013506': 0.5}     0.0     0.0   
 

In [313]:
fl_f_teams = fl_f.Title.str.extract(r'^(.+) vs. (.+)$')
print(fl_f_teams.groupby(fl_f_teams.columns.to_list()).size().to_string())

0                    1                  
Alliance             Chaos Esports Club      8
                     Keen Gaming             8
                     Mineski                12
                     Newbee                  8
                     Royal Never Give Up     4
Chaos Esports Club   Keen Gaming             8
                     Mineski                 8
Evil Geniuses        Fnatic                  8
                     Infamous                8
                     Natus Vincere           8
                     Ninjas in Pyjamas       4
                     OG                      8
                     Royal Never Give Up     8
                     Team Liquid             8
Fnatic               Infamous                8
                     Natus Vincere           6
                     Team Liquid             4
Infamous             Keen Gaming             4
                     Team Secret            10
Keen Gaming          Mineski                 8
Mineski            

In [315]:
pred_win_prob = match_pred.map_win_prob(
    fl_f_teams[0],
    fl_f_teams[1],
    (fl_f.Descr.str[0].astype(int) - 1),
    fl_f.ClosD)
pred_win_prob.head()

pred_win_prob team_1_wins            map_start_time
ID           wager_type                                                     
177218520623 against          0.704767        True 2019-08-15 02:03:30+00:00
             on               0.704767        True 2019-08-15 02:03:30+00:00
177218529861 against          0.722984        True 2019-08-15 03:16:37+00:00
             on               0.722984        True 2019-08-15 03:16:37+00:00
177218614562 against          0.602035       False 2019-08-15 02:02:17+00:00

### Assign the computed win probability and compute EVs.

In [316]:
fl_f = fl_f.assign(
    pred_win_prob=pred_win_prob.iloc[:, 0],
    team_1_won=pred_win_prob.iloc[:, 1],
    map_start=pred_win_prob.iloc[:, 2].dt.tz_convert('US/Eastern'))
fl_f['time_diff'] = (fl_f.map_start - fl_f.ClosD) / pd.Timedelta(minutes=1)
fl_f['ev'] = np.where(fl_f.wager_type == 'on',
                      fl_f.pred_win_prob * fl_f.odds - 1,
                      (1 - fl_f.pred_win_prob) * fl_f.odds - 1)
fl_f['outcome'] = np.where(
    fl_f.team_1_won.isna(),
    np.nan,
    np.where(np.logical_or(np.logical_and(fl_f.wager_type == 'on',
                                          fl_f.team_1_won),
                           np.logical_and(fl_f.wager_type == 'against',
                                          np.logical_not(fl_f.team_1_won))),
             fl_f.odds - 1,
             -1))
fl_f

Comp                     Title  \
ID           wager_type                                                         
177218520623 against     Dota 2 - The International  Team Secret vs. Alliance   
             on          Dota 2 - The International  Team Secret vs. Alliance   
177218529861 against     Dota 2 - The International  Team Secret vs. Alliance   
...                                             ...                       ...   
178196223735 on          Dota 2 - The International        OG vs. Team Liquid   
178199751032 against     Dota 2 - The International        OG vs. Team Liquid   
             on          Dota 2 - The International        OG vs. Team Liquid   

                           Descr dota_market_type                LastSoftCh  \
ID           wager_type                                                       
177218520623 against     1st Map              map 2019-08-14 04:17:17-04:00   
             on          1st Map              map 2019-08-13 23:51:32-04:00   
177218529861 against     2nd Map              map 2019-08-14 04:17:17-04:00   
...                          ...              ...                       ...   
178196223735 on          3rd Map              map 2019-08-25 07:00:00-04:00   
178199751032 against     4th Map              map 2019-08-25 07:59:59-04:00   
             on          4th Map              map 2019-08-25 07:59:59-04:00   

                                            ClosD wager_type   RunnerName  \
ID           wager_type                                                     
177218520623 against    2019-08-14 20:59:51-04:00    against  Team Secret   
             on         2019-08-14 20:59:51-04:00         on  Team Secret   
177218529861 against    2019-08-14 20:59:51-04:00    against  Team Secret   
...                                           ...        ...          ...   
178196223735 on         2019-08-25 07:27:03-04:00         on           OG   
178199751032 against    2019-08-25 08:25:50-04:00    against           OG   
             on         2019-08-25 08:25:50-04:00         on           OG   

                         handicap   odds  amount    winp  winp_c  \
ID           wager_type                                            
177218520623 against          0.0  3.538   75.11  0.7174  0.7169   
             on               0.0  1.414   69.11  0.7072  0.7077   
177218529861 against          0.0  3.564   76.11  0.7194  0.7189   
...                           ...    ...     ...     ...     ...   
178196223735 on               0.0  1.751  638.11  0.5711  0.5717   
178199751032 against          0.0  2.479  710.11  0.5967  0.5961   
             on               0.0  1.499  960.11  0.6671  0.6677   

                         RunnerVolMatched  CatID                    SettlD  \
ID           wager_type                                                      
177218520623 against                  0.0     32 2019-08-14 21:59:51-04:00   
             on                       0.0     32 2019-08-14 21:59:51-04:00   
177218529861 against                  0.0     32 2019-08-14 21:59:51-04:00   
...                                   ...    ...                       ...   
178196223735 on                       0.0     32 2019-08-25 07:33:03-04:00   
178199751032 against                  0.0     32 2019-08-25 08:31:50-04:00   
             on                       0.0     32 2019-08-25 08:31:50-04:00   

                         Status  _Type  _Period  SettlT    Comm  \
ID           wager_type                                           
177218520623 against          0      0        0       0  0.0025   
             on               0      0        0       0  0.0025   
177218529861 against          0      0        0       0  0.0025   
...                         ...    ...      ...     ...     ...   
178196223735 on               1      0        0       0  0.0025   
178199751032 against          1      0        0       0  0.0025   
             on               1      0        0       0  0.

In [317]:
fl_f.to_csv('ti9_groups_ev_outcome.20190829.tsv', sep='\t')